In [ ]:

import pandas as pd
import numpy as np
import os
INPUT_FOLDER = 'Regression_data' 
OUTPUT_FOLDER = 'Price_Capped_Aggregated_Data' 
TRAIN_PRICE_FILE = 'train_price_final.csv'
TEST_PRICE_FILE = 'test_price_final.csv'
TRAIN_PRICE_OUT_FILE = 'train_price_capped_agg.csv' 
TEST_PRICE_OUT_FILE = 'test_price_capped_agg.csv'  
TARGET_COL = 'Price' 
AREA_COL = '面积_数值' 
ID_COLUMN = 'ID'
GROUPING_COLS = ['板块', '区域', '区县']
LOWER_PERCENTILE = 0.01
UPPER_PERCENTILE = 0.99
COLS_TO_CAP = [TARGET_COL, AREA_COL] 

try:
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f"输出文件夹 '{OUTPUT_FOLDER}' 已创建或已存在。")
except OSError as e:
    print(f"创建文件夹 '{OUTPUT_FOLDER}' 时出错: {e}")
    exit()

print(f"\n--- 从 '{INPUT_FOLDER}' 加载 Price 数据 ---")
try:
    train_price_df = pd.read_csv(os.path.join(INPUT_FOLDER, TRAIN_PRICE_FILE), encoding='utf-8-sig')
    test_price_df = pd.read_csv(os.path.join(INPUT_FOLDER, TEST_PRICE_FILE), encoding='utf-8-sig')
    print("Price 数据加载成功。")
    print(f"  原始训练集形状: {train_price_df.shape}")
    print(f"  原始测试集形状: {test_price_df.shape}")
except FileNotFoundError as e:
    print(f"加载 Price 数据时出错: {e}. 请确保文件路径正确。")
    exit()
except Exception as e:
    print(f"加载 Price 数据时发生其他错误: {e}")
    exit()

print(f"\n--- 步骤 1: 对 '{', '.join(COLS_TO_CAP)}' 应用 {LOWER_PERCENTILE*100}% / {UPPER_PERCENTILE*100}% Capping ---")
bounds = {}
print("  计算 Capping 边界 (基于训练集)...")
cols_actually_capped = [] 
for col in COLS_TO_CAP:
    if col not in train_price_df.columns: 
        print(f"  警告: 列 '{col}' 不在训练集中，跳过 Capping。")
        continue
    try:
        lower_bound = train_price_df[col].quantile(LOWER_PERCENTILE)
        upper_bound = train_price_df[col].quantile(UPPER_PERCENTILE)
        if lower_bound < upper_bound:
            bounds[col] = {'lower': lower_bound, 'upper': upper_bound}
            print(f"    - 列 '{col}': 下界={lower_bound:.2f}, 上界={upper_bound:.2f}")
            cols_actually_capped.append(col) # 记录下来
        else:
            print(f"  警告: 列 '{col}' 的百分位数相同，不应用 Capping。")
    except Exception as e:
        print(f"  计算列 '{col}' 边界时出错: {e}")

print("\n  应用 Capping...")
capped_count_train = {col: 0 for col in bounds}
capped_count_test = {col: 0 for col in bounds}

# 应用到训练集
for col in cols_actually_capped:
    if col in bounds:
        b = bounds[col]
        original_series = train_price_df[col].copy()
        train_price_df[col] = train_price_df[col].clip(lower=b['lower'], upper=b['upper'])
        capped_count_train[col] = ((original_series < b['lower']) | (original_series > b['upper'])).sum()
print(f"  训练集 Capping 统计: {capped_count_train}")
# 应用到测试集 (使用训练集边界)
for col in cols_actually_capped:
     if col in bounds and col in test_price_df.columns: # 检查 test_price_df
        b = bounds[col]
        original_series = test_price_df[col].copy()
        test_price_df[col] = test_price_df[col].clip(lower=b['lower'], upper=b['upper'])
        capped_count_test[col] = ((original_series < b['lower']) | (original_series > b['upper'])).sum()
     elif col not in test_price_df.columns:
         print(f"  警告: 列 '{col}' 不在测试集中，无法应用 Capping。")
print(f"  测试集 Capping 统计: {capped_count_test}")
print("--- Capping 完成 ---")

print(f"\n--- 步骤 2: 基于 Capped Price 数据计算和合并聚合特征 ---")
aggregation_done_price = False # 变量名区分
# 检查必需列 (分组列和 Capped 列)
required_cols_agg_price = GROUPING_COLS + cols_actually_capped 
if not all(col in train_price_df.columns for col in required_cols_agg_price) or \
   not all(col in test_price_df.columns for col in GROUPING_COLS):
    print(f"错误: Capped Price 数据缺少聚合所需的列。跳过聚合特征。")
else:
    aggregation_done_price = True
    # 计算全局统计量 (基于 capped 训练集)
    print("  计算全局统计量 (基于 capped Price 训练集)...")
    global_mean_price = train_price_df[TARGET_COL].mean() if TARGET_COL in cols_actually_capped else np.nan
    global_median_price = train_price_df[TARGET_COL].median() if TARGET_COL in cols_actually_capped else np.nan
    global_mean_area = train_price_df[AREA_COL].mean() if AREA_COL in cols_actually_capped else np.nan
    print(f"    - 全局平均价格: {global_mean_price:.2f}, 全局中位数价格: {global_median_price:.2f}, 全局平均面积: {global_mean_area:.2f}")
    # 循环处理每个分组列
    for group_col in GROUPING_COLS:
        print(f"\n  正在处理分组: '{group_col}'")
        agg_funcs = {}
        if TARGET_COL in cols_actually_capped:
             agg_funcs[f'{group_col}_平均价格_capped'] = pd.NamedAgg(column=TARGET_COL, aggfunc='mean')
             agg_funcs[f'{group_col}_中位数价格_capped'] = pd.NamedAgg(column=TARGET_COL, aggfunc='median')
        if AREA_COL in cols_actually_capped:
             agg_funcs[f'{group_col}_平均面积_capped'] = pd.NamedAgg(column=AREA_COL, aggfunc='mean')
        if not agg_funcs:
            print(f"    - 没有可用于 '{group_col}' 分组聚合的 capped 列，跳过。")
            continue
        agg_stats_price = train_price_df.groupby(group_col).agg(**agg_funcs).reset_index() # 变量名区分
        print(f"    - 计算了 {len(agg_stats_price)} 个分组的统计量。")
        # 合并 (合并到已经 capped 的 DataFrame)
        try:
            train_price_df = pd.merge(train_price_df, agg_stats_price, on=group_col, how='left')
            test_price_df = pd.merge(test_price_df, agg_stats_price, on=group_col, how='left')
            print(f"    - 已合并统计量到 train_price_df 和 test_price_df。")
        except Exception as e:
            print(f"    - 合并 '{group_col}' 聚合特征时出错: {e}")
            aggregation_done_price = False
            continue
        # 填充 NaN (基于 capped 全局统计量)
        fillna_cols_price = list(agg_funcs.keys()) # 获取实际创建的列名
        fill_values_price = {} # 变量名区分
        if f'{group_col}_平均价格_capped' in fillna_cols_price: fill_values_price[f'{group_col}_平均价格_capped'] = global_mean_price
        if f'{group_col}_中位数价格_capped' in fillna_cols_price: fill_values_price[f'{group_col}_中位数价格_capped'] = global_median_price
        if f'{group_col}_平均面积_capped' in fillna_cols_price: fill_values_price[f'{group_col}_平均面积_capped'] = global_mean_area
        nan_test = test_price_df[fillna_cols_price].isna().sum().sum()
        if nan_test > 0:
             print(f"    - 正在填充 test_price_df 中的 {nan_test} 个 NaN 值...")
             test_price_df.fillna(fill_values_price, inplace=True)
        nan_train = train_price_df[fillna_cols_price].isna().sum().sum()
        if nan_train > 0:
             print(f"    - 正在填充 train_price_df 中的 {nan_train} 个 NaN 值...")
             train_price_df.fillna(fill_values_price, inplace=True)
    if aggregation_done_price:
        print("\n聚合特征工程执行完毕。")
    else:
        print("\n聚合特征工程未能完全成功。")
print("\n--- 步骤 1 & 2: Capping 和 聚合特征完成 ---")

print(f"\n--- 保存 Capped 和 Aggregated 的 Price 数据到 '{OUTPUT_FOLDER}' ---")
try:
    train_output_path = os.path.join(OUTPUT_FOLDER, TRAIN_PRICE_OUT_FILE)
    test_output_path = os.path.join(OUTPUT_FOLDER, TEST_PRICE_OUT_FILE)
    train_price_df.to_csv(train_output_path, index=False, encoding='utf-8-sig')
    test_price_df.to_csv(test_output_path, index=False, encoding='utf-8-sig')
    print(f"✓ 处理后的训练集已保存: {train_output_path} (新形状: {train_price_df.shape})")
    print(f"✓ 处理后的测试集已保存: {test_output_path} (新形状: {test_price_df.shape})")
except Exception as e:
    print(f"保存文件时出错: {e}")
print("\n--- 脚本执行完毕 ---")

输出文件夹 'Price_Capped_Aggregated_Data' 已创建或已存在。

--- 从 'Regression_data' 加载 Price 数据 ---
Price 数据加载成功。
  原始训练集形状: (103871, 187)
  原始测试集形状: (34017, 187)

--- 步骤 1: 对 'Price, 面积_数值' 应用 1.0% / 99.0% Capping ---
  计算 Capping 边界 (基于训练集)...
    - 列 'Price': 下界=254478.05, 上界=12990538.99
    - 列 '面积_数值': 下界=28.97, 上界=281.56

  应用 Capping...
  训练集 Capping 统计: {'Price': 2078, '面积_数值': 2078}
  警告: 列 'Price' 不在测试集中，无法应用 Capping。
  测试集 Capping 统计: {'Price': 0, '面积_数值': 289}
--- Capping 完成 ---

--- 步骤 2: 基于 Capped Price 数据计算和合并聚合特征 ---
  计算全局统计量 (基于 capped Price 训练集)...
    - 全局平均价格: 2210077.95, 全局中位数价格: 1479407.11, 全局平均面积: 98.72

  正在处理分组: '板块'
    - 计算了 977 个分组的统计量。
    - 已合并统计量到 train_price_df 和 test_price_df。
    - 正在填充 test_price_df 中的 639 个 NaN 值...

  正在处理分组: '区域'
    - 计算了 117 个分组的统计量。
    - 已合并统计量到 train_price_df 和 test_price_df。
    - 正在填充 test_price_df 中的 315 个 NaN 值...

  正在处理分组: '区县'
    - 计算了 110 个分组的统计量。
    - 已合并统计量到 train_price_df 和 test_price_df。
    - 正在填充 test_price_df 中的 147 个 NaN